In [1]:
#Importing required libraries

from pymongo import MongoClient
from bson.objectid import ObjectId
import pandas as pd
from datetime import datetime, timedelta
import random
from faker import Faker

In [50]:
# Connect to the local MongoDB server and select the 'eduhub_db' database
client = MongoClient("mongodb://localhost:27017/")
db = client["eduhub_db"]

# Define collection references for easier access
users = db["users"]
courses = db["courses"]
lesson = db["lesson"]
assignment = db["assignment"]
assignment_submission = db["assignment_submission"]
enrollment = db["enrollment"]


In [3]:
#Create a faker instance
fake = Faker()

In [4]:
user_schema = {

    "_id": ObjectId(),
    
    "user_id": "user_1",

    "email": "first.last@example.com",

    "first_name": "first",

    "last_name": "last",

    "role": "student",

    "date_joined": datetime.now(),

    "profile": {

        "bio": "easy_life",

        "avatar": "stay_wicked",

        "skills": "engineering"

    },

    "is_active": True

}

Part 3: Basic CRUD Operations 

In [48]:
# find one document
def find_document(col, query, projection=None):
    try:
        res = col.find_one(query, projection)
        return res
    except Exception as e:
        print(f"Error finding document: {e}")
        return None

#find many documents
def find_many_documents(col, query, projection=None):
    try:
        result = []
        res = col.find(query, projection)
        for doc in res:
            result.append(doc)
        return result
    except Exception as e:
        print(f"Error finding many documents: {e}")
        return None

def insert_one(col, document):
    try:
        res = col.insert_one(document)
    except Exception as e:
        print(f"error inserting: {e}")
        return None
    return res.inserted_id

def delete_document(col, query):
    try:
        res = col.delete_one(query)
    except Exception as e:
        print(f"error deleting: {e}")
    return res.deleted_count

def insert_many(col, documents):
    try:
        res = col.insert_many(documents)
        return len(res.inserted_ids)
    except Exception as e:
        print(f"error inserting: {e}")
        return 0
    
def delete_many_document(col, query):
    try:
        res = col.delete_many(query)
        return res.deleted_count
    except Exception as e:
        print(f"error deleting: {e}")
        return 0
    
#update one document
def update_document(col, query, update):
    try:
        result = col.update_one(query, update)
        return result.modified_count
    except Exception as e:
        print(f"Error updating document: {e}")
        return None

#update many documents
def update_many_documents(col, query, update):
    try:
        result = col.update_many(query, update)
        return result.modified_count
    except Exception as e:
        print(f"Error updating many documents: {e}")
        return None

AGGREGATION FUNCTION

In [6]:
def aggregate(col, query):
    try:
        result = []
        resp= col.aggregate(query)
        for res in resp:
            result.append(res)
        print(len(result))
        return result
    except Exception as e:
        print(f"Error updating document: {e}")
        return None


INDEX FUNCTION


In [7]:
def create_index_safe(col, field_name, unique=False, order=1):
    try:
        index_name = col.create_index([(field_name, order)], unique=unique)
        print(f" Index '{index_name}' created on '{field_name}' field.")
        return index_name
    except Exception as e:
        print(f" Error creating index: {e}")
        return None


In [8]:
# Data pools for randomization
first_names = [
    "Ade", "Chinedu", "Funke", "Grace", "Ibrahim", "Kemi", "Olu", "Tunde", 
    "Ngozi", "Chika", "Emeka", "Bola", "Yusuf", "Amina", "Eze", "Fatima",
    "Obinna", "Zainab", "Segun", "Halima", "Chukwu", "Adanna", "Mohammed", "Temitope",
    "Kunle", "Amara", "Sani", "Ifeoma", "Okonkwo", "Zara", "Jide", "Deborah",
    "Musa", "Chidinma", "Wale", "Patience", "Abdul", "Chioma", "Femi", "Blessing"
]

last_names = [
    "Adebayo", "Chukwu", "Eze", "Okafor", "Sule", "Yusuf", "Balogun", "Okoro",
    "Mohammed", "Ogunleye", "Ibrahim", "Nwosu", "Adeyemi", "Bello", "Okafor",
    "Mustapha", "Obi", "Aliyu", "Akintola", "Onyema", "Salami", "Umar", "Olu",
    "Bakare", "Okeke", "Adewale", "Ojo", "Adeleke", "Oladipo", "Akinwande",
    "Onyeka", "Alade", "Okonkwo", "Onyeama", "Afolayan", "Okorie", "Onyia", "Uche",
    "Okoh", "Ogbonna"
]

avatar = ["stay_wicked", "blessed_assurance", "warmly_endurance", "brutal_tenancious"]

interests = ["reading", "writing", "coding", "designing", "marketing", "sales", "management", "finance",
             "hr", "it", "networking", "security", "database", "cloud", "devops", "blockchain", 
            "iot", "mobile", "cybersecurity", "big data", "data visualization", "ui/ux design", "blockchain", "cybersecurity", "data science", 
            "data analysis", "data engineering", "data visualization", "data mining", "data warehousing", "data modeling", "data cleaning", "data integration",
            "data transformation", "data loading", "data storage", "data retrieval", "data processing", "data analysis", "data mining", "data warehousing",
            "data modeling", "data cleaning", "data integration", "data transformation", "data loading", "data storage", "data retrieval", "data processing"]

USER COLLECTION SCHEMA

In [ ]:
# Example document structure for the 'users' collection
user_schema = {

    "_id": ObjectId(),                # Unique identifier automatically generated by MongoDB

    "user_id": "user_1",              # Custom user identifier (for reference in other collections)

    "email": "first.last@example.com",# User's email address

    "first_name": "first",            # User's first name
    "last_name": "last",              # User's last name

    "role": "student",                # User role (e.g., student, instructor, admin)

    "date_joined": datetime.now(),    # Date the user registered or joined

    "profile": {                      # Nested profile details
        "bio": "easy_life",           # Short personal or professional description
        "avatar": "stay_wicked",      # Avatar name or image reference
        "skills": "engineering"       # User’s skill or area of expertise
    },

    "is_active": True                 # Indicates if the user account is active
}


20 users (mix of students and instructors)

In [9]:
# Generate sample user documents
users_list = []
for i in range(1, 21):
    first_name = random.choice(first_names)
    last_name = random.choice(last_names)

    users_dict = {
        "_id": ObjectId(),  # Unique MongoDB ID
        "user_id": f"user_{i}",
        "first_name": first_name,
        "last_name": last_name,
        "email": f"{first_name.lower()}.{last_name.lower()}@eduhub.com",
        "role": "student" if i < 17 else "instructor",  # First 16 are students, last 4 are instructors
        "date_joined": datetime.now(),
        "profile": {
            "bio": f"I am {first_name}, based in {fake.city()} with interest in {random.choice(interests)} and {random.choice(interests)}",
            "avatar": random.choice(avatar),
            "skills": random.sample(interests, 4)
        },
        "is_active": random.choice([True, False, True])  # Randomly set account as active/inactive
    }

    users_list.append(users_dict)




In [10]:
# Insert all user documents into the 'users' collection
insert_many(users, users_list)


error inserting: batch op errors occurred, full error: {'writeErrors': [{'index': 5, 'code': 11000, 'errmsg': 'E11000 duplicate key error collection: eduhub_db.users index: email_1 dup key: { email: "blessing.yusuf@eduhub.com" }', 'keyPattern': {'email': 1}, 'keyValue': {'email': 'blessing.yusuf@eduhub.com'}, 'op': {'_id': ObjectId('68e2dc56c3defe6ef29302d8'), 'user_id': 'user_6', 'first_name': 'Blessing', 'last_name': 'Yusuf', 'email': 'blessing.yusuf@eduhub.com', 'role': 'student', 'date_joined': datetime.datetime(2025, 10, 5, 22, 0, 6, 899034), 'profile': {'bio': 'I am Blessing, based in West Larryshire with interest in ui/ux design and data warehousing', 'avatar': 'blessed_assurance', 'skills': ['writing', 'blockchain', 'designing', 'blockchain']}, 'is_active': True}}], 'writeConcernErrors': [], 'nInserted': 5, 'nUpserted': 0, 'nMatched': 0, 'nModified': 0, 'nRemoved': 0, 'upserted': []}


0

COURSE COLLECTION SCHEMA

In [11]:
# Example document structure for the 'courses' collection
course_schema = {
    "_id": ObjectId(),                           # Unique MongoDB document ID
    "course_id": "course_1",                     # Custom course identifier
    "title": "databases",                        # Course title
    "description": "introduction to databases",  # Short course description
    "instructorId": "user_17",                   # Reference to the instructor (from users collection)
    "category": random.choice(["cs", "data", "math", "cloud"]),  # Course category
    "level": random.choice(["beginner", "intermediate", "advanced"]),  # Difficulty level
    "duration": random.choice([6, 8, 10, 12]),   # Duration in weeks
    "price": random.choice([0, 29, 49, 79]),     # Course price
    "tag": random.sample(["mongo", "python", "etl", "devops", "cloud", "math"], 3),  # Related tags
    "createdAt": datetime.now(),                 # Creation timestamp
    "updatedAt": datetime.now(),                 # Last update timestamp
    "isPublished": random.choice([True, True, False])  # Publication status
}


In [12]:
# Filter and store all users with the role 'instructor' from the users list
instructors = [user for user in users_list if user["role"] == "instructor"]


8 courses across different categories


In [13]:
course_titles = [
    "Intro to Databases", "Aggregation Piplines", "Python for Data",
    "Discrete Math", "NoSql", "Algorithm 101",
    "Devops Basics", "Cloud Fundamentals"
]

course_list = []

for i, course_title in enumerate(course_titles):
    instructor = random.choice(instructors)

    # generate a random float between 1.0 and 5.0 with 1 decimal place
    rating = round(random.uniform(3.0, 5.0), 1)

    course_dict ={
    "_id": ObjectId(),
    "course_id": f"course_{i+1}",
    "title": course_titles[i],
    "description": f"introduction to {course_titles[i]}",
    "instructorId": instructor["user_id"],
    "category": random.choice(["cs", "data", "math", "cloud"]),
    "level": random.choice(["beginner", "intermediate", "advanced"]),
    "duration":random.choice([6, 8, 10,12]),
    "price": random.choice([0,29, 49, 79]),
    "tag": random.sample(["mongo", "python", "etl", "devops", "cloud", "math"], 3),
    "rating": rating, 
    "createdAt": datetime.now(),
    "updatedAt": datetime.now(),
    "isPublished":random.choice([True, True, False]),
    

    }
    
    course_list.append(course_dict)

In [14]:
# Insert all course documents into the 'courses' collection
insert_many(courses, course_list)


8

LESSON COLLECTION SCHEMA

In [15]:
# Example document structure for the 'lesson' collection
lesson_schema = {
    "_id": ObjectId(),                          # Unique MongoDB document ID
    "lesson_id": "lesson_1",                    # Custom lesson identifier
    "course_id": "course_1",                    # Reference to the related course
    "instructor_id": "instructor_1",            # Reference to the instructor teaching the lesson
    "title": "exploring mongodb",               # Lesson title
    "topics": ["indexes", "collection", "queries"],  # List of topics covered in the lesson
    "description": "This lesson introduces mongodb"  # Brief summary of the lesson content
}


Logic to create 25 lessons

In [16]:


# Pool of possible lesson topics
topics_pool = [
    "Introduction", "Core Concepts", "Hands-on Practice",
    "Advanced Techniques", "Case Study", "Troubleshooting",
    "Best Practices", "Optimization", "Assessment", "Review"
]

lesson_list = []

# Generate three lessons for each course
for course in course_list:
    for n in range(3):
        topic = random.choice(topics_pool)
        lesson_dict = {
            "_id": ObjectId(),
            "lesson_id": f"lesson_{len(lesson_list)+1}",
            "course_id": course["course_id"],
            "instructor_id": course["instructorId"],
            "title": f"exploring {course['title']} - {topic}",
            "topic": topic,
            "resources_url": "hhtps://contentname@eduhub.com",
            "description": f"This lesson covers {topic} in {course['title']}"
        }
        lesson_list.append(lesson_dict)

# Add one extra lesson randomly
course = random.choice(course_list)
topic = random.choice(topics_pool)

lesson_dict = {
    "_id": ObjectId(),
    "lesson_id": f"lesson_{len(lesson_list)+1}",
    "course_id": course["course_id"],
    "instructor_id": course["instructorId"],
    "title": f"exploring {course['title']} - {topic}",
    "topic": topic,
    "resources_url": "hhtps://contentname@eduhub.com",
    "description": f"This lesson covers {topic} in {course['title']}"
}

lesson_list.append(lesson_dict)

print(len(lesson_list))






25


In [17]:
# Insert all lesson documents into the 'lesson' collection
insert_many(lesson, lesson_list)


25

ASSIGNMENT COLLECTION SCHEMA

In [18]:
# Example document structure for the 'assignment' collection
assignment_schema = {
    "_id": ObjectId(),                                        # Unique MongoDB document ID
    "assignment_id": "assignment_1",                          # Custom assignment identifier
    "student_id": "student_1",                                # Reference to the assigned student
    "lesson_id": "lesson_1",                                  # Reference to the related lesson
    "course_id": "course_1",                                  # Reference to the related course
    "due_date": datetime(2025, 10, 30),                       # Assignment submission deadline
    "description": "This assignment test on introduction to databases",  # Brief description
    "grade": "B+",                                            # Letter grade received
    "grade_point": 3.3                                        # Numeric grade equivalent
}


Logic go create 10 assignments

In [19]:
# # 
assignment_list_pre = []
for i, lesson in enumerate(lesson_list):

    # Mapping of letter grades to grade points
    grade_scale = {"A": 4.0, "A-": 3.7, "B+": 3.3, "B": 3.0, "B-": 2.7, "C+": 2.3, "C": 2.0, "D": 1.0, "F": 0.0}

    # Pick grade before creating the dictionary
    grade = random.choice(list(grade_scale.keys()))

    assignment_dict = {
        "_id": ObjectId(),
        "assignment_id": f"assignment_{i+1}",
        "student_id": [f"student{i+1}" for i, user in enumerate(users_list) if user["role"] == "student"],  # Linked students
        "lesson_id": lesson["lesson_id"],            # Reference to the related lesson
        "course_id": lesson_list[i]["course_id"],    # Reference to the related course
        "due_date": datetime.now() + timedelta(days=random.randint(0, 30)),  # Random due date within 30 days
        "description": f"This assignment test on {lesson['topic']}",         # Assignment description
        "grade": grade,                   # Random grade selection
        "grade_point": grade_scale[grade]                 # Numeric grade value
    }

    assignment_list_pre.append(assignment_dict)

# Limit to the first 10 assignments
assignment_list = assignment_list_pre[:10]

print(assignment_list)
# Generate assignment documents based on existing lessons
assignment_list_pre = []



[{'_id': ObjectId('68e2dc57c3defe6ef293030b'), 'assignment_id': 'assignment_1', 'student_id': ['student1', 'student2', 'student3', 'student4', 'student5', 'student6', 'student7', 'student8', 'student9', 'student10', 'student11', 'student12', 'student13', 'student14', 'student15', 'student16'], 'lesson_id': 'lesson_1', 'course_id': 'course_1', 'due_date': datetime.datetime(2025, 11, 2, 22, 0, 7, 53322), 'description': 'This assignment test on Hands-on Practice', 'grade': 'F', 'grade_point': 0.0}, {'_id': ObjectId('68e2dc57c3defe6ef293030c'), 'assignment_id': 'assignment_2', 'student_id': ['student1', 'student2', 'student3', 'student4', 'student5', 'student6', 'student7', 'student8', 'student9', 'student10', 'student11', 'student12', 'student13', 'student14', 'student15', 'student16'], 'lesson_id': 'lesson_2', 'course_id': 'course_1', 'due_date': datetime.datetime(2025, 10, 12, 22, 0, 7, 53339), 'description': 'This assignment test on Review', 'grade': 'D', 'grade_point': 1.0}, {'_id': O

In [20]:
# Insert all assignment documents into the 'assignment' collection
insert_many(assignment, assignment_list)


10

ASSIGNMENT SUBMISSION COLLECTION SCHEMA

In [21]:
# Example document structure for the 'assignment_submission' collection
assignment_submission_schema = {
    "_id": ObjectId(),                                               # Unique MongoDB document ID
    "submission_id": "submission_1",                                 # Custom submission identifier
    "description": "This assignment should be submitted in PDF format before the due date",  # Submission instructions
    "submission_date": datetime(2025, 10, 30),                       # Date of submission
    "isSubmitted": True                                              # Indicates if the assignment was submitted
}


Logic to create 12 assignment submissions

In [138]:
# Generate sample assignment submissions
assignment_sub_list = []


# for i, assignment in enumerate(assignment_list_pre[:12]):
for i in range(12):
    # Randomize submission date (some before, some after due date)
    submission_date = datetime(2025, 10, 30) - timedelta(days=random.choice([1, 2, 3, 4, -1, -2]))

    assignment_sub_dict = {
        "_id": ObjectId(),                                         # Unique MongoDB document ID
        "submission_id": f"submission_{i+1}",                      # Custom submission identifier
        "description": "This assignment should be submitted in PDF format before the due date",
        "submission_date": submission_date,                        # Actual date of submission
        "isSubmitted": True if submission_date <= datetime(2025, 10, 30) else False,  # Submission status
        "submission_link": "submission@eduhub.com"                 # Placeholder link for submission
    }

  

    assignment_sub_list.append(assignment_sub_dict)

print(len(assignment_sub_list))  # Print total submissions created


12


In [139]:
# Insert all assignment submission documents into the 'assignment_submission' collection
insert_many(assignment_submission, assignment_sub_list)


12

ENROLLMENT SCHEMA

In [90]:
# Example document structure for the 'enrollment' collection
enrollment_schema = {
    "_id": ObjectId(),                               # Unique MongoDB document ID
    "enrollment_id": "enrollment_1",                 # Custom enrollment identifier
    "course_id": "course_1",                         # Reference to the enrolled course
    "student_id": "user_1",                          # Reference to the student
    "enrollment_date": datetime(2024, 10, 30),       # Date of enrollment
    "is_active": True                                # Indicates if the enrollment is currently active
}


In [91]:
# Filter and store all users with the role 'student'
students_list = [user for user in users_list if user["role"] == "student"]

enrollment_list = []

# Generate enrollment records for students
for i, student in enumerate(students_list):
    enrollment_dict = {
        "_id": ObjectId(),                                            # Unique MongoDB document ID
        "enrollment_id": f"enrollment_{i+1}",                         # Custom enrollment identifier
        "course_id": random.choice(course_list)["course_id"],          # Randomly assign a course to the student
        "student_id": f"user_{i+1}",                                  # Reference to the student
        "enrollment_date": datetime(2024, 10, 30) - timedelta(days=random.randint(1, 30)),  # Random enrollment date
        "is_active": random.choice([True, True, True, False])          # Random active/inactive status
    }

    enrollment_list.append(enrollment_dict)

# Keep only the first 15 enrollments
enrollment_list = enrollment_list[:15]

print(len(enrollment_list))  # Print total number of enrollments created


15


In [92]:
# Insert all enrollment documents into the 'enrollment' collection
insert_many(enrollment, enrollment_list)


15

Part 3: Basic CRUD Operations 

Task 3.1: Create Operations

3.1.1 - Insert a new User

In [93]:
#Create a new single user document to be added to the collection
new_user = {
    "_id": ObjectId(),
    "user_id": "user_21",
    "first_name": "Tosin",
    "last_name": "Adebayo",
    "email": "tosin.adebayo@eduhub.com",
    "role": "student",  
    "date_joined": datetime(2023, 5, 20),
    "profile": {
        "bio": "I am Tosin, based in London with interest in databases and data analysis.",
        "avatar": "curious_owl",
        "skills": ["python", "mongodb", "data modeling", "etl"]
    },
    "is_active": True
}




In [94]:
#Insert new user into user collection
insert_one(users, new_user)

error inserting: E11000 duplicate key error collection: eduhub_db.users index: email_1 dup key: { email: "tosin.adebayo@eduhub.com" }, full error: {'index': 0, 'code': 11000, 'errmsg': 'E11000 duplicate key error collection: eduhub_db.users index: email_1 dup key: { email: "tosin.adebayo@eduhub.com" }', 'keyPattern': {'email': 1}, 'keyValue': {'email': 'tosin.adebayo@eduhub.com'}}


3.1.2 - Create a new course


In [95]:
# Create a new single course document to be added to the collection
instructor = random.choice(instructors)
rating = round(random.uniform(3.0, 5.0), 1)

new_course = {
    "_id": ObjectId(),
    "course_id": "course_9",
    "title": "Data Visualization Fundamentals",
    "description": "Introduction to data visualization tools and principles using Python and modern libraries.",
    "instructorId": instructor["user_id"],
    "category": "data",
    "level": "beginner",
    "duration": 8,
    "price": 49,
    "tag": ["python", "data", "visualization"],
    "rating": rating,
    "createdAt": datetime.now(),
    "updatedAt": datetime.now(),
    "isPublished": True
}


In [96]:
#Insert new course document into course collection
insert_one(courses, new_course)

ObjectId('68e2e45bc3defe6ef2930359')

3.1.3 - Enroll a new student

In [97]:
#Create a new single enrollment document to be added to the collection
new_enrollment = {
    "_id": ObjectId(),
    "enrollment_id": "enrollment_16",   
    "course_id": random.choice(course_list)["course_id"], 
    "student_id": "student_16",        
    "enrollment_date": datetime(2024, 10, 30) - timedelta(days=random.randint(1, 30)),
    "is_active": True
}

In [98]:
#Insert a new enrollment document into enrollment collection
insert_one(enrollment, new_enrollment)

ObjectId('68e2e45bc3defe6ef293035a')

3.1.4 - Add a new lesson to an existing course

In [99]:
#Add a new lesson to an existing course
new_lesson = {
    "_id": ObjectId(),
    "lesson_id": f"lesson_{len(lesson_list)+1}",
    "course_id": course["course_id"],         
    "instructor_id": course["instructorId"],
    "title": f"Exploring {course['title']} - {topic}",
    "topic": topic,
    "resources_url": "https://contentname@eduhub.com",   
    "description": f"This lesson covers {topic} in {course['title']}"
}


In [100]:
#Insert a new lesson document into lesson collection
insert_one(lesson, new_lesson)

ObjectId('68e2e45bc3defe6ef293035b')

Task 3.2: Read Operations

3.2.1 - Find all active students

In [101]:
# Find and display all active users with the role 'student'
query = {"role": "student", "is_active": True}
result = find_many_documents(users, query)

for doc in result:
    print(doc)


{'_id': ObjectId('68e28c3743a8ff36f4d1e6cd'), 'user_id': 'user_4', 'first_name': 'Olu', 'last_name': 'Ogbonna', 'email': 'olu.ogbonna@eduhub.com', 'role': 'student', 'date_joined': datetime.datetime(2025, 10, 5, 16, 18, 15, 675000), 'profile': {'bio': 'I am Olu, based in Harmonfurt with interest in iot and data processing', 'avatar': 'brutal_tenancious', 'skills': ['data loading', 'data processing', 'data analysis', 'data visualization']}, 'is_active': True}
{'_id': ObjectId('68e28c3743a8ff36f4d1e6ce'), 'user_id': 'user_5', 'first_name': 'Yusuf', 'last_name': 'Aliyu', 'email': 'yusuf.aliyu@eduhub.com', 'role': 'student', 'date_joined': datetime.datetime(2025, 10, 5, 16, 18, 15, 675000), 'profile': {'bio': 'I am Yusuf, based in Ashleymouth with interest in coding and data visualization', 'avatar': 'stay_wicked', 'skills': ['data loading', 'data mining', 'data transformation', 'sales']}, 'is_active': True}
{'_id': ObjectId('68e28c3743a8ff36f4d1e6d1'), 'user_id': 'user_8', 'first_name': '

3.2.2 - Retrieve course details with instructor information

In [102]:
# Retrieve course details along with instructor information using aggregation
pipeline = [
    {
        "$lookup": {                             # Join 'courses' with 'users' collection
            "from": "users",
            "localField": "instructorId",        # Field in 'courses' collection
            "foreignField": "user_id",           # Matching field in 'users' collection
            "as": "instructor_info"              # Output array field containing matched instructor details
        }
    },
    {"$unwind": "$instructor_info"},             # Flatten the joined instructor info
    {
        "$project": {                            # Select specific fields to return
            "_id": 0,
            "course_id": 1,
            "title": 1,
            "category": 1,
            "level": 1,
            "price": 1,
            "instructor_info.first_name": 1,
            "instructor_info.last_name": 1,
            "instructor_info.email": 1
        }
    }
]

# Execute the aggregation and print each course with instructor details
results = list(courses.aggregate(pipeline))
for course in results:
    print(course)


{'course_id': 'course_1', 'title': 'Intro to Databases', 'category': 'cs', 'level': 'advanced', 'price': 49, 'instructor_info': {'first_name': 'Jide', 'last_name': 'Balogun', 'email': 'jide.balogun@eduhub.com'}}
{'course_id': 'course_1', 'title': 'Intro to Databases', 'category': 'cs', 'level': 'advanced', 'price': 49, 'instructor_info': {'first_name': 'Segun', 'last_name': 'Adewale', 'email': 'segun.adewale@eduhub.com'}}
{'course_id': 'course_2', 'title': 'Aggregation Piplines', 'category': 'math', 'level': 'intermediate', 'price': 29, 'instructor_info': {'first_name': 'Chidinma', 'last_name': 'Mohammed', 'email': 'chidinma.mohammed@eduhub.com'}}
{'course_id': 'course_2', 'title': 'Aggregation Piplines', 'category': 'math', 'level': 'intermediate', 'price': 29, 'instructor_info': {'first_name': 'Kunle', 'last_name': 'Chukwu', 'email': 'kunle.chukwu@eduhub.com'}}
{'course_id': 'course_3', 'title': 'Python for Data', 'category': 'math', 'level': 'beginner', 'price': 49, 'instructor_info

3.2.3 - Get all courses in a specific category

In [103]:
# Find and display all courses within the 'data' category
query = {"category": "data"}
result = find_many_documents(courses, query)

for course in result:
    print(course)


{'_id': ObjectId('68e293e943a8ff36f4d1e700'), 'course_id': 'course_9', 'title': 'Data Visualization Fundamentals', 'description': 'Introduction to data visualization tools and principles using Python and modern libraries.', 'instructorId': 'user_20', 'category': 'data', 'level': 'beginner', 'duration': 8, 'price': 49, 'tag': ['python', 'data', 'visualization'], 'rating': 4.9, 'createdAt': datetime.datetime(2025, 10, 5, 16, 51, 5, 829000), 'updatedAt': datetime.datetime(2025, 10, 5, 16, 51, 5, 829000), 'isPublished': True}
{'_id': ObjectId('68e2d95cd9abc9eb13d086c6'), 'course_id': 'course_2', 'title': 'Aggregation Piplines', 'description': 'introduction to Aggregation Piplines', 'instructorId': 'user_17', 'category': 'data', 'level': 'advanced', 'duration': 8, 'price': 29, 'tag': ['cloud', 'math', 'etl'], 'rating': 3.2, 'createdAt': datetime.datetime(2025, 10, 5, 21, 47, 24, 934000), 'updatedAt': datetime.datetime(2025, 10, 5, 21, 47, 24, 934000), 'isPublished': False}
{'_id': ObjectId(

3.2.4 - Find students enrolled in a particular course

In [104]:
# Retrieve all students enrolled in a specific course
course_id_to_find = "course_1"

pipeline = [
    {"$match": {"course_id": course_id_to_find}},  # Filter enrollments for the selected course
    {
        "$lookup": {                               # Join 'enrollment' with 'users' collection
            "from": "users",
            "localField": "student_id",            # Field in 'enrollment' collection
            "foreignField": "user_id",             # Matching field in 'users' collection
            "as": "student_info"                   # Output array with matched student details
        }
    },
    {"$unwind": "$student_info"},                  # Flatten the joined student information
    {
        "$project": {                              # Select relevant fields to display
            "_id": 0,
            "enrollment_id": 1,
            "course_id": 1,
            "student_info.first_name": 1,
            "student_info.last_name": 1,
            "student_info.email": 1
        }
    }
]

# Run the aggregation pipeline on the 'enrollment' collection
aggregate(enrollment, pipeline)


12


[{'enrollment_id': 'enrollment_6',
  'course_id': 'course_1',
  'student_info': {'first_name': 'Grace',
   'last_name': 'Bello',
   'email': 'grace.bello@eduhub.com'}},
 {'enrollment_id': 'enrollment_6',
  'course_id': 'course_1',
  'student_info': {'first_name': 'Segun',
   'last_name': 'Yusuf',
   'email': 'segun.yusuf@eduhub.com'}},
 {'enrollment_id': 'enrollment_9',
  'course_id': 'course_1',
  'student_info': {'first_name': 'Amina',
   'last_name': 'Adewale',
   'email': 'amina.adewale@eduhub.com'}},
 {'enrollment_id': 'enrollment_9',
  'course_id': 'course_1',
  'student_info': {'first_name': 'Mohammed',
   'last_name': 'Adewale',
   'email': 'mohammed.adewale@eduhub.com'}},
 {'enrollment_id': 'enrollment_10',
  'course_id': 'course_1',
  'student_info': {'first_name': 'Okonkwo',
   'last_name': 'Okeke',
   'email': 'okonkwo.okeke@eduhub.com'}},
 {'enrollment_id': 'enrollment_10',
  'course_id': 'course_1',
  'student_info': {'first_name': 'Sani',
   'last_name': 'Onyema',
   'em

3.2.5 - Search courses by title (case-insensitive, partial match)

In [105]:
# Find a course by its title and return selected fields only
query = {"title": "NoSql"}

projection = {
    "course_id": 1,
    "title": 1,
    "category": 1,
    "level": 1
}

# Retrieve matching course documents with the specified projection
find_many_documents(courses, query, projection=projection)


[{'_id': ObjectId('68e293c543a8ff36f4d1e6fc'),
  'course_id': 'course_5',
  'title': 'NoSql',
  'category': 'cs',
  'level': 'intermediate'},
 {'_id': ObjectId('68e2d95cd9abc9eb13d086c9'),
  'course_id': 'course_5',
  'title': 'NoSql',
  'category': 'cs',
  'level': 'intermediate'},
 {'_id': ObjectId('68e2dc56c3defe6ef29302ec'),
  'course_id': 'course_5',
  'title': 'NoSql',
  'category': 'data',
  'level': 'beginner'}]

Task 3.3: Update Operations

3.3.1 - Update a user’s profile information


In [106]:
# Update the last name of a user whose first name is 'Halima'
query = {"first_name": "Halima"}
update = {"$set": {"last_name": "Chinedu"}}

# Apply the update operation to the 'users' collection
update_document(users, query, update)


0

3.3.2 - Mark a course as published

In [107]:
# Publish a specific course by updating its 'isPublished' status
query = {"course_id": "course_2"}
update = {"$set": {"isPublished": True}}

# Apply the update to the 'courses' collection
update_document(courses, query, update)


0

3.3.3 - Update assignment grades


In [108]:
# Update all assignments with grade 'B-' to 'B+'
query = {"grade": "B-"}
update = {"$set": {"grade": "B+"}}

# Apply the update to all matching documents in the 'assignment' collection
update_many_documents(assignment, query, update)


0

3.3.4 - Add tags to an existing course

In [109]:
# Add a new tag 'database' to the course with ID 'course_1' (avoiding duplicates)
query = {"course_id": "course_1"}
update = {"$addToSet": {"tag": "database"}}

# Apply the update to the 'courses' collection
update_document(courses, query, update)


0

Task 3.4: Delete Operations

3.4.1 - Remove a user (soft delete by setting isActive to false)

In [110]:
# Deactivate a user by setting 'is_active' to False for user_1
query = {"user_id": "user_1"}
update = {"$set": {"is_active": False}}

# Apply the update to the 'users' collection
update_document(users, query, update)


0

3.4.2 - Delete an enrollment

In [111]:
# Delete a specific enrollment record with ID 'enrollment_1'
query = {"enrollment_id": "enrollment_1"}

# Remove the matching document from the 'enrollment' collection
delete_document(enrollment, query)


1

3.4.3 - Remove a lesson from a course

In [112]:
# Delete a specific lesson document with ID 'lesson_1'
query = {"lesson_id": "lesson_1"}

# Remove the matching document from the 'lesson' collection
delete_document(lesson, query)


1

Part 4: Advanced Queries and Aggregation

Task 4.1: Complex Queries
Write Python code using various PyMongo operators:

4.1.1 - Find courses with price between $50 and $200

In [113]:
# Find and display all courses priced between $50 and $200
query = {"price": {"$gte": 50, "$lte": 200}}

projection = {
    "course_id": 1,
    "title": 1,
    "price": 1
}

# Retrieve matching courses with selected fields
result = find_many_documents(courses, query, projection=projection)

for doc in result:
    print(doc)


{'_id': ObjectId('68e293c543a8ff36f4d1e6fc'), 'course_id': 'course_5', 'title': 'NoSql', 'price': 79}
{'_id': ObjectId('68e2d95cd9abc9eb13d086c8'), 'course_id': 'course_4', 'title': 'Discrete Math', 'price': 79}
{'_id': ObjectId('68e2d95cd9abc9eb13d086cc'), 'course_id': 'course_8', 'title': 'Cloud Fundamentals', 'price': 79}
{'_id': ObjectId('68e2dc56c3defe6ef29302eb'), 'course_id': 'course_4', 'title': 'Discrete Math', 'price': 79}


4.1.2 - Get users who joined in the last 6 months

In [114]:
# calculate 6 months ago (approx 30 days × 6)
six_months_ago = datetime.now() - timedelta(days=180)

query = { "date_joined": {"$gt": six_months_ago}}

result = find_many_documents(users, query)
for doc in result:
    print(doc)

{'_id': ObjectId('68e28c3743a8ff36f4d1e6ca'), 'user_id': 'user_1', 'first_name': 'Musa', 'last_name': 'Onyia', 'email': 'musa.onyia@eduhub.com', 'role': 'student', 'date_joined': datetime.datetime(2025, 10, 5, 16, 18, 15, 675000), 'profile': {'bio': 'I am Musa, based in Davidton with interest in data analysis and data processing', 'avatar': 'blessed_assurance', 'skills': ['data warehousing', 'it', 'data integration', 'designing']}, 'is_active': False}
{'_id': ObjectId('68e28c3743a8ff36f4d1e6cb'), 'user_id': 'user_2', 'first_name': 'Kunle', 'last_name': 'Balogun', 'email': 'kunle.balogun@eduhub.com', 'role': 'student', 'date_joined': datetime.datetime(2025, 10, 5, 16, 18, 15, 675000), 'profile': {'bio': 'I am Kunle, based in West Stacy with interest in reading and data warehousing', 'avatar': 'brutal_tenancious', 'skills': ['it', 'data retrieval', 'blockchain', 'hr']}, 'is_active': False}
{'_id': ObjectId('68e28c3743a8ff36f4d1e6cc'), 'user_id': 'user_3', 'first_name': 'Adanna', 'last_na

4.1.3 - Find courses that have specific tags using $in operator

In [115]:
# Find and display all courses that include either 'mongo' or 'python' in their tags
query = {"tag": {"$in": ["mongo", "python"]}}

projection = {
    "_id": 0,
    "course_id": 1,
    "title": 1,
    "tag": 1
}

# Retrieve matching courses with selected fields
result = find_many_documents(courses, query, projection)

for doc in result:
    print(doc)


{'course_id': 'course_1', 'title': 'Intro to Databases', 'tag': ['devops', 'etl', 'mongo', 'database']}
{'course_id': 'course_3', 'title': 'Python for Data', 'tag': ['mongo', 'etl', 'devops']}
{'course_id': 'course_4', 'title': 'Discrete Math', 'tag': ['etl', 'cloud', 'python']}
{'course_id': 'course_6', 'title': 'Algorithm 101', 'tag': ['python', 'math', 'mongo']}
{'course_id': 'course_8', 'title': 'Cloud Fundamentals', 'tag': ['python', 'mongo', 'cloud']}
{'course_id': 'course_9', 'title': 'Data Visualization Fundamentals', 'tag': ['python', 'data', 'visualization']}
{'course_id': 'course_1', 'title': 'Intro to Databases', 'tag': ['python', 'mongo', 'cloud']}
{'course_id': 'course_3', 'title': 'Python for Data', 'tag': ['math', 'devops', 'python']}
{'course_id': 'course_4', 'title': 'Discrete Math', 'tag': ['mongo', 'python', 'devops']}
{'course_id': 'course_6', 'title': 'Algorithm 101', 'tag': ['math', 'python', 'etl']}
{'course_id': 'course_7', 'title': 'Devops Basics', 'tag': ['et

4.1.4 - Retrieve assignments with due dates in the next week

In [116]:
# Find and display assignments with due dates within the next 7 days
today = datetime.now()
next_week = today + timedelta(days=7)

# Query assignments due between today and next week
query = {
    "due_date": {
        "$gte": today,
        "$lte": next_week
    }
}

projection = {
    "assignment_id": 1,
    "course_id": 1,
    "due_date": 1,
    "description": 1
}

# Retrieve matching assignments with selected fields
result = find_many_documents(assignment, query, projection=projection)

for doc in result:
    print(doc)

 


{'_id': ObjectId('68e296c643a8ff36f4d1e707'), 'assignment_id': 'assignment_7', 'course_id': 'course_3', 'due_date': datetime.datetime(2025, 10, 8, 17, 3, 18, 217000), 'description': 'This assignment test on Advanced Techniques'}
{'_id': ObjectId('68e2dc57c3defe6ef293030f'), 'assignment_id': 'assignment_5', 'course_id': 'course_2', 'due_date': datetime.datetime(2025, 10, 8, 22, 0, 7, 53000), 'description': 'This assignment test on Hands-on Practice'}
{'_id': ObjectId('68e296c643a8ff36f4d1e701'), 'assignment_id': 'assignment_1', 'course_id': 'course_1', 'due_date': datetime.datetime(2025, 10, 10, 17, 3, 18, 217000), 'description': 'This assignment test on Troubleshooting'}
{'_id': ObjectId('68e2dc57c3defe6ef2930311'), 'assignment_id': 'assignment_7', 'course_id': 'course_3', 'due_date': datetime.datetime(2025, 10, 10, 22, 0, 7, 53000), 'description': 'This assignment test on Troubleshooting'}
{'_id': ObjectId('68e2dc57c3defe6ef2930312'), 'assignment_id': 'assignment_8', 'course_id': 'cou

Task 4.2: Aggregation Pipeline
Create aggregation pipelines using PyMongo

4.2 - Course Enrollment Statistics:

4.2.1 - Count total enrollments per course

In [117]:
# Count total enrollments per course using aggregation
pipeline = [
    {
        "$group": {                                   # Group documents by course_id
            "_id": "$course_id",
            "total_enrollments": {"$sum": 1}          # Count number of enrollments in each course
        }
    },
    {
        "$project": {                                 # Format the output
            "_id": 0,                                 # Exclude default _id field
            "course_id": "$_id",                      # Rename _id to course_id
            "total_enrollments": 1                    # Include total_enrollments field
        }
    },
    {
        "$sort": {"total_enrollments": -1}            # Sort by enrollment count (descending)
    }
]

# Execute aggregation and print results
results = aggregate(enrollment, pipeline)
for doc in results:
    print(doc)


8
{'total_enrollments': 9, 'course_id': 'course_2'}
{'total_enrollments': 7, 'course_id': 'course_5'}
{'total_enrollments': 6, 'course_id': 'course_1'}
{'total_enrollments': 6, 'course_id': 'course_8'}
{'total_enrollments': 5, 'course_id': 'course_3'}
{'total_enrollments': 5, 'course_id': 'course_4'}
{'total_enrollments': 4, 'course_id': 'course_7'}
{'total_enrollments': 3, 'course_id': 'course_6'}


4.2.2 - Calculate average course rating

In [118]:
# Calculate the average rating for each course
pipeline = [
    {"$group": {"_id": "$course_id", "avg_rating": {"$avg": "$rating"}}},  # Group by course_id and compute average rating
    {"$project": {"_id": 0, "course_id": "$_id", "avg_rating": {"$round": ["$avg_rating", 2]}}},  # Format and round result
    {"$sort": {"avg_rating": -1}}  # Sort courses by average rating (highest first)
]

# Execute aggregation and display the results
results = aggregate(courses, pipeline)
for doc in results:
    print(doc)


9
{'course_id': 'course_9', 'avg_rating': 4.37}
{'course_id': 'course_3', 'avg_rating': 4.23}
{'course_id': 'course_6', 'avg_rating': 4.1}
{'course_id': 'course_4', 'avg_rating': 4.03}
{'course_id': 'course_5', 'avg_rating': 3.97}
{'course_id': 'course_7', 'avg_rating': 3.97}
{'course_id': 'course_2', 'avg_rating': 3.7}
{'course_id': 'course_8', 'avg_rating': 3.67}
{'course_id': 'course_1', 'avg_rating': 3.63}


4.2.3 - Group by course category

In [119]:
# Group courses by category and count how many exist in each category
pipeline = [
    {
        "$group": {                                # Group documents by category
            "_id": "$category",
            "total_courses": {"$sum": 1}           # Count number of courses per category
        }
    },
    {
        "$project": {                              # Format the output
            "_id": 0,                              # Exclude default _id field
            "category": "$_id",                    # Rename _id to category
            "total_courses": 1                     # Include total_courses field
        }
    },
    {
        "$sort": {"total_courses": -1}             # Sort categories by number of courses (descending)
    }
]

# Execute aggregation and print results
results = aggregate(courses, pipeline)
for doc in results:
    print(doc)


4
{'total_courses': 14, 'category': 'math'}
{'total_courses': 6, 'category': 'data'}
{'total_courses': 6, 'category': 'cs'}
{'total_courses': 1, 'category': 'cloud'}


4.2 - Student Performance Analysis:


4.2.2 - Average grade per student

In [120]:
# Calculate the average grade point (GPA) for each student
pipeline = [
    {
        "$group": {                                        # Group assignments by student_id
            "_id": "$student_id",
            "avg_grade_point": {"$avg": "$grade_point"}    # Compute average grade_point per student
        }
    },
    {
        "$project": {                                      # Format the output
            "_id": 0,
            "student_id": "$_id",
            "avg_grade_point": {"$round": ["$avg_grade_point", 2]}  # Round to 2 decimal places
        }
    },
    {"$sort": {"avg_grade_point": -1}}                     # Sort by GPA in descending order
]

# Execute aggregation and print results
results = aggregate(assignment, pipeline)
for doc in results:
    print(doc)


1
{'student_id': ['student1', 'student2', 'student3', 'student4', 'student5', 'student6', 'student7', 'student8', 'student9', 'student10', 'student11', 'student12', 'student13', 'student14', 'student15', 'student16'], 'avg_grade_point': 2.14}


4.2.3 - Completion rate by course

In [121]:
# Calculate course completion rate based on enrollment data
pipeline = [
    {
        "$group": {                                              # Group enrollments by course_id
            "_id": "$course_id",
            "total_enrollments": {"$sum": 1},                    # Count total enrollments
            "completed_count": {                                 # Count how many students completed the course
                "$sum": {
                    "$cond": [{"$eq": ["$is_active", False]}, 1, 0]  # If is_active = False, count as completed
                }
            }
        }
    },
    {
        "$project": {                                            # Format the output fields
            "_id": 0,
            "course_id": "$_id",
            "total_enrollments": 1,
            "completed_count": 1,
            "completion_rate": {                                 # Calculate completion rate as a percentage
                "$concat": [
                    {
                        "$toString": {
                            "$round": [
                                {"$multiply": [
                                    {"$divide": ["$completed_count", "$total_enrollments"]}, 100
                                ]},
                                2
                            ]
                        }
                    },
                    "%"
                ]
            }
        }
    },
    {"$sort": {"completion_rate": -1}}                           # Sort by completion rate in descending order
]

# Execute aggregation and print results
results = aggregate(enrollment, pipeline)

for result in results:
    print(result)


8
{'total_enrollments': 5, 'completed_count': 3, 'course_id': 'course_4', 'completion_rate': '60%'}
{'total_enrollments': 3, 'completed_count': 1, 'course_id': 'course_6', 'completion_rate': '33.33%'}
{'total_enrollments': 9, 'completed_count': 3, 'course_id': 'course_2', 'completion_rate': '33.33%'}
{'total_enrollments': 6, 'completed_count': 2, 'course_id': 'course_1', 'completion_rate': '33.33%'}
{'total_enrollments': 4, 'completed_count': 1, 'course_id': 'course_7', 'completion_rate': '25%'}
{'total_enrollments': 7, 'completed_count': 1, 'course_id': 'course_5', 'completion_rate': '14.29%'}
{'total_enrollments': 5, 'completed_count': 0, 'course_id': 'course_3', 'completion_rate': '0%'}
{'total_enrollments': 6, 'completed_count': 0, 'course_id': 'course_8', 'completion_rate': '0%'}


4.2.4 - Top-performing students

In [122]:
# Identify the top-performing students based on their average grade points
pipeline = [
    {
        "$group": {                                           # Group assignments by student_id
            "_id": "$student_id",
            "average_grade": {"$avg": "$grade_point"},        # Calculate each student's average grade
            "assignments_count": {"$sum": 1}                  # Count total assignments per student
        }
    },
    {
        "$project": {                                         # Format the output
            "_id": 0,
            "student_id": "$_id",
            "average_grade": {"$round": ["$average_grade", 2]},  # Round average grade to 2 decimals
            "assignments_count": 1
        }
    },
    {"$sort": {"average_grade": -1}},                        # Sort students by average grade (highest first)
    {"$limit": 5}                                            # Return only the top 5 students
]

# Execute aggregation and display results
results = aggregate(assignment, pipeline)
for result in results:
    print(result)



1
{'assignments_count': 20, 'student_id': ['student1', 'student2', 'student3', 'student4', 'student5', 'student6', 'student7', 'student8', 'student9', 'student10', 'student11', 'student12', 'student13', 'student14', 'student15', 'student16'], 'average_grade': 2.14}


4.3 - Instructor Analytics:


4.3.1 - Total students taught by each instructor

In [123]:
# Calculate the total number of unique students taught by each instructor
pipeline = [
    # Join 'courses' with 'enrollment' collection to connect instructors with their enrolled students
    {
        "$lookup": {
            "from": "enrollment",
            "localField": "course_id",     # Match by course_id in 'courses'
            "foreignField": "course_id",   # Match by course_id in 'enrollment'
            "as": "enrollments"            # Store matching documents in 'enrollments' array
        }
    },

    # Flatten the enrollments array so each enrollment becomes a separate document
    {"$unwind": "$enrollments"},

    # Group by instructorId to aggregate all students under each instructor
    {
        "$group": {
            "_id": "$instructorId",
            "unique_students": {"$addToSet": "$enrollments.student_id"}  # Collect unique student IDs per instructor
        }
    },

    # Calculate total number of unique students per instructor
    {
        "$project": {
            "_id": 0,
            "instructorId": "$_id",
            "total_students": {"$size": "$unique_students"}  # Count how many unique students each instructor has
        }
    },

    # Sort instructors by total student count in descending order
    {"$sort": {"total_students": -1}}
]

# Execute aggregation and display results
results = aggregate(courses, pipeline)

for result in results:
    print(result)


4
{'instructorId': 'user_17', 'total_students': 15}
{'instructorId': 'user_19', 'total_students': 15}
{'instructorId': 'user_18', 'total_students': 15}
{'instructorId': 'user_20', 'total_students': 13}


4.3.2 - Average course rating per instructor


In [124]:
# Calculate the average course rating per instructor
pipeline = [
    {
        "$group": {                                      # Group courses by instructorId
            "_id": "$instructorId",
            "avg_rating": {"$avg": "$rating"},           # Calculate each instructor’s average rating
            "total_courses": {"$sum": 1}                 # Count total courses taught by the instructor
        }
    },
    {
        "$project": {                                    # Format the output
            "_id": 0,
            "instructorId": "$_id",
            "avg_rating": {"$round": ["$avg_rating", 2]},  # Round average rating to 2 decimal places
            "total_courses": 1
        }
    },
    {"$sort": {"avg_rating": -1}}                       # Sort instructors by rating in descending order
]



In [125]:
results = aggregate(courses, pipeline)
for result in results:
    print(result)

4
{'total_courses': 6, 'instructorId': 'user_18', 'avg_rating': 4.3}
{'total_courses': 6, 'instructorId': 'user_20', 'avg_rating': 4.03}
{'total_courses': 5, 'instructorId': 'user_19', 'avg_rating': 3.8}
{'total_courses': 10, 'instructorId': 'user_17', 'avg_rating': 3.8}


4.3.3 - Revenue generated per instructor

In [126]:
# Calculate total revenue, total courses, and average course price per instructor
pipeline = [

    # Group all courses by instructorId
    {
        "$group": {
            "_id": "$instructorId",
            "total_revenue": {"$sum": "$price"},           # Sum of all course prices per instructor
            "total_courses": {"$sum": 1},                  # Count total number of courses per instructor
            "avg_course_price": {"$avg": "$price"}         # Compute average course price
        }
    },
    
    # Format and round the output values
    {
        "$project": {
            "_id": 0,
            "instructorId": "$_id",
            "total_revenue": {"$round": ["$total_revenue", 2]},     # Round revenue to 2 decimals
            "total_courses": 1,
            "avg_course_price": {"$round": ["$avg_course_price", 2]} # Round average price to 2 decimals
        }
    },
    
    # Sort instructors by total revenue in descending order
    {"$sort": {"total_revenue": -1}}
]

# Execute the aggregation and print results
results = aggregate(courses, pipeline)
for result in results:
    print(result)


4
{'total_courses': 6, 'instructorId': 'user_20', 'total_revenue': 305, 'avg_course_price': 50.83}
{'total_courses': 6, 'instructorId': 'user_18', 'total_revenue': 206, 'avg_course_price': 34.33}
{'total_courses': 10, 'instructorId': 'user_17', 'total_revenue': 205, 'avg_course_price': 20.5}
{'total_courses': 5, 'instructorId': 'user_19', 'total_revenue': 186, 'avg_course_price': 37.2}


4.4 - Advanced Analytics:

4.4.1 - Monthly enrollment trends

In [127]:
# Analyze monthly enrollment trends over time
pipeline = [
    # Extract year and month from enrollment_date
    {
        "$addFields": {
            "year": {"$year": "$enrollment_date"},
            "month": {"$month": "$enrollment_date"}
        }
    },

    # Group by year and month to count enrollments per period
    {
        "$group": {
            "_id": {"year": "$year", "month": "$month"},
            "total_enrollments": {"$sum": 1}
        }
    },

    # Sort results by year and month to show chronological trend
    {
        "$sort": {
            "_id.year": 1,
            "_id.month": 1
        }
    },

    # Format output for readability
    {
        "$project": {
            "_id": 0,
            "year": "$_id.year",
            "month": "$_id.month",
            "total_enrollments": 1
        }
    }
]

# Execute aggregation and print monthly enrollment results
results = aggregate(enrollment, pipeline)

for result in results:
    print(result)



1
{'total_enrollments': 45, 'year': 2024, 'month': 10}


4.4.2 - Most popular course categories


In [128]:
# Identify the most popular course categories based on enrollment data
pipeline = [
    # Join 'enrollment' with 'courses' to access category information
    {
        "$lookup": {
            "from": "courses",
            "localField": "course_id",          # Match course_id in 'enrollment'
            "foreignField": "course_id",        # With course_id in 'courses'
            "as": "course_info"                 # Store matched course details
        }
    },

    # Flatten the joined course_info array
    {"$unwind": "$course_info"},

    # Group by course category and count total enrollments per category
    {
        "$group": {
            "_id": "$course_info.category",
            "total_enrollments": {"$sum": 1}
        }
    },

    # Sort categories by total enrollments (highest first)
    {"$sort": {"total_enrollments": -1}},

    # Limit results to top three most popular categories
    {"$limit": 3},

    # Format the output for readability
    {
        "$project": {
            "_id": 0,
            "categories": "$_id",
            "total_enrollments": 1
        }
    }
]

# Execute aggregation and display top categories
results = aggregate(enrollment, pipeline)
for result in results:
    print(result)




3
{'total_enrollments': 75, 'categories': 'math'}
{'total_enrollments': 35, 'categories': 'cs'}
{'total_enrollments': 21, 'categories': 'data'}


4.4.3 - Student engagement metrics

In [129]:
# Measure student engagement based on assignment submission rates
pipeline = [
    # Group assignments by student_id
    {
        "$group": {
            "_id": "$student_id",
            "total_assignments": {"$sum": 1},                         # Count all assignments
            "submitted_assignments": {                                # Count only submitted ones
                "$sum": {"$cond": [{"$eq": ["$isSubmitted", True]}, 1, 0]}
            }
        }
    },

    # Calculate engagement rate as a percentage of submitted assignments
    {
        "$project": {
            "_id": 0,
            "student_id": "$_id",
            "total_assignments": 1,
            "submitted_assignments": 1,
            "engagement_rate": {                                      # (submitted / total) * 100
                "$round": [
                    {
                        "$multiply": [
                            {"$divide": ["$submitted_assignments", "$total_assignments"]},
                            100
                        ]
                    },
                    2                                                 # Round to 2 decimal places
                ]
            }
        }
    },

    # Sort students by engagement rate (highest first)
    {"$sort": {"engagement_rate": -1}}
]

# Execute aggregation and print engagement results
results = aggregate(assignment, pipeline)
for result in results:
    print(result)



1
{'total_assignments': 20, 'submitted_assignments': 0, 'student_id': ['student1', 'student2', 'student3', 'student4', 'student5', 'student6', 'student7', 'student8', 'student9', 'student10', 'student11', 'student12', 'student13', 'student14', 'student15', 'student16'], 'engagement_rate': 0.0}


Part 5: Indexing and Performance

Task 5.1: Index Creation

5.1.1 - User email lookup

In [130]:
# Create a unique index on the 'email' field in the 'users' collection to prevent duplicates
create_index_safe(users, "email", unique=True)



 Index 'email_1' created on 'email' field.


'email_1'

In [131]:
print(users.index_information())

{'_id_': {'v': 2, 'key': [('_id', 1)]}, 'email_1': {'v': 2, 'key': [('email', 1)], 'unique': True}}


5.1.2 - Course search by title and category

In [132]:
# Create an index on the 'title' field to speed up course title searches
create_index_safe(courses, "title")

# Create an index on the 'category' field to optimize queries filtering by category
create_index_safe(courses, "category")


 Index 'title_1' created on 'title' field.
 Index 'category_1' created on 'category' field.


'category_1'

In [133]:
print(courses.index_information())

{'_id_': {'v': 2, 'key': [('_id', 1)]}, 'category_1_title_1': {'v': 2, 'key': [('category', 1), ('title', 1)]}, 'title_1': {'v': 2, 'key': [('title', 1)]}, 'category_1': {'v': 2, 'key': [('category', 1)]}}


5.1.3 - Assignment queries by due date

In [134]:
# Create an index on the 'due_date' field to improve performance of assignment deadline queries
create_index_safe(assignment, "due_date")



 Index 'due_date_1' created on 'due_date' field.


'due_date_1'

In [135]:
print(assignment.index_information())


{'_id_': {'v': 2, 'key': [('_id', 1)]}, 'due_date_1': {'v': 2, 'key': [('due_date', 1)]}}


5.1.4 - Enrollment queries by student and course

In [136]:
# Create an index on 'student_id' to optimize queries filtering enrollments by student
create_index_safe(enrollment, "student_id")

# Create an index on 'course_id' to speed up queries retrieving enrollments per course
create_index_safe(enrollment, "course_id")


 Index 'student_id_1' created on 'student_id' field.
 Index 'course_id_1' created on 'course_id' field.


'course_id_1'